# Ejemplo de como crear y dibujar la grilla a utilizar en el problema de tomografía de la tarea

## GF5013 - Metodos Inversos Aplicados a la Geofisica
## Primavera 2019
Prof. Francisco Hernán Ortega Culaciati 

ortega.francisco@u.uchile.cl

Departamento de Geofísica - FCFM

Universidad de Chile

Octubre de 2019


In [1]:
import sys, os  # CAMBIE EL pathTarea2 indicando la ruta completa de la carpeta Tarea2_NombreAlumno en su computador.
pathTarea2 = '../'
pathPaquetesPython = pathTarea2 + '/PaquetesPython'
TomoDataFilename = pathTarea2 + '/datosTomo/datosTomografia.txt'

sys.path.append(os.path.abspath(pathPaquetesPython))

In [2]:
import matplotlib
%matplotlib notebook 


import matplotlib.pyplot as plt
import numpy as NP
# descomentar esto es para hacer los plots afuera del notebook
#%matplotlib tk  
from Tarea2 import Mesh
from Tarea2.TomoData import readTravelTimeData as readTTdata
from Tarea2 import TomographyModel as Tmodel

## Ejemplo para definir malla genérica

In [ ]:
Xmin = -20
Xmax = 20
Nx = 4
Ymin = 10
Ymax = 90
Ny = 8

RM = Mesh.RectangularMesh(Xmin = Xmin, Xmax = Xmax, Nx = Nx, Ymin = Ymin, Ymax = Ymax, Ny = Ny)


## Dibujar Indices de la Malla

In [ ]:
fig = plt.figure(1)
axes = []
axes.append( fig.add_subplot(121) )
axes.append( fig.add_subplot(122) )

Mesh.plotRectMeshIndices(RM, fig, axes[0], element = True, IndexFontSize = 10, \
                         xlabel = 'X', ylabel = 'Y', title = 'IDs de Elementos',\
                         axequal = True)
Mesh.plotRectMeshIndices(RM, fig, axes[1], element = False, IndexFontSize = 9, \
                         xlabel = 'X', ylabel = '', title = 'Índices de Elementos',\
                         axequal = True)


## Dibujar un campo escalar en la malla


In [ ]:
# field to plot is the number of neighbors for each elements ID
F = NP.array([ len(Elem_Neighbors) for Elem_Neighbors in RM.neighbors.values()  ])
fig = plt.figure(10)
fig.clear()
ax = plt.subplot(111)
fig, ax = Mesh.plotRectMeshField(F, RM, fig, ax , colorlim = None, \
                                 title = "Campo es # de vecinos del Elemento",\
                                 colorbar_label = 'Number of Neighbors', xlabel = 'X', ylabel = 'Y', colormap = 'Spectral_r')
fig.show()


In [ ]:
F = NP.random.rand(len(RM.ID),1) * 10 # field to plot is random numbers
colorlim = [0, 10]
colormap = 'jet'
fig = plt.figure(11)
fig.clear()
ax = plt.subplot(111)
fig, ax = Mesh.plotRectMeshField(F, RM, fig, ax , colorlim = colorlim, \
                                 title = "Campo Aleatorio ",\
                                 colorbar_label = 'bla', xlabel = 'X', ylabel = 'Y', \
                                 axequal = True, colormap = colormap)
fig.show()

## Ejemplo en que se grafica la malla para la tarea con un campo constante, y encima se grafica los rayos rectos a considerar en el experimento de tomografía

In [ ]:
# LEER DATOS TOMOGRAFIA
TTdata = readTTdata(TomoDataFilename)
# Coordenadas Fuentes
xF = TTdata['xF']
zF = TTdata['zF']
# Coordenadas Receptores
xR = TTdata['xR']
zR = TTdata['zR']
# Tiempo de Viaje
TV = TTdata['TV']

# Calcular parametros necesarios para definir la malla y instanciar el objeto RectangularMesh
Xmin = 0 
Xmax = 100
Zmin = -200
Zmax = 0

Nx = 10
Nz = 20
# inicializar Tmesh (Tomography Mesh) como un Rectangular Mesh
Tmesh = Mesh.RectangularMesh(Xmin = Xmin, Xmax = Xmax, Nx = Nx, \
                             Ymin = Zmin, Ymax = Zmax, Ny = Nz)

In [ ]:
# graficar malla con indice

fig = plt.figure(30)
axes = []
axes.append( fig.add_subplot(111) )

Mesh.plotRectMeshIndices(Tmesh, fig, axes[0], element = True, IndexFontSize = 8, \
                         xlabel = 'X [m]', ylabel = 'Z [m]', title = 'IDs de Elementos',\
                         axequal = True)

In [ ]:
# graficar malla con rayos del problema de tomografía
F = NP.ones(len(Tmesh.ID)) * 0.0 # field to plot is random numbers
colorlim = [-1, 1]
colormap = 'seismic'
plotColorbar = False

fig = plt.figure(31)
axes = []
axes.append( fig.add_subplot(111) )

# graficar malla
title = 'Problema de Tomografía \n (Rayos se asumen rectos)'
fig, ax = Mesh.plotRectMeshField(F, Tmesh, fig, axes[0] , colorlim = colorlim, \
                                 title = title, \
                                 colorbar = plotColorbar, xlabel = 'X [m]', ylabel = 'Z [m]', \
                                 axequal = True, colormap = colormap, \
                                 edgewidth = 2, edgecolor = 'gray')

# graficar rayos
for k in range(0, len(xF)):
    # dibujar rayos
    color = 'green'
    if k == 0:
        ax.plot([xF[k], xR[k]], [zF[k], zR[k]], '-', linewidth = 0.75, color = color,\
                label = 'Rayos' )
    else:
        ax.plot([xF[k], xR[k]], [zF[k], zR[k]], '-', linewidth = 0.75, color = color )
# dibujar fuente y receptor
ax.plot(xF, zF, 'or', markersize = 4, label = 'Fuentes')
ax.plot(xR, zR, 'ob', markersize = 4, label = 'Receptores')
ax.legend()
fig.show()

## Plot a ray in the mesh along with the length on each mesh element

In [ ]:
# select ray
rID = 17 # ray ID
fractional_precision = 1E-3

# compute ray and lengths
ray = Tmodel.RayLinear2D(xF[rID], zF[rID], xR[rID], zR[rID])
L, dR = ray.calcMeshRayLength(Tmesh, fractional_precision)

colorlim = None
colormap = 'hot_r'
plotColorbar = True

fig = plt.figure(32)
axes = []
axes.append( fig.add_subplot(111) )

# graficar malla
title = 'Longitud del rayo ' + str(rID) + ' en cada elemento de la malla \n (precision = '+ str(dR) +' [m])'
fig, ax = Mesh.plotRectMeshField(L, Tmesh, fig, axes[0] , colorlim = colorlim, \
                                 title = title, \
                                 colorbar = plotColorbar, xlabel = 'X [m]', ylabel = 'Z [m]', \
                                 axequal = True, colormap = colormap, \
                                 edgewidth = 1, edgecolor = 'gray', colorbar_label='Length [m]')

# plot ray
ax.plot([ray.xS, ray.xR], [ray.yS, ray.yR], 'b-o', )

fig.show()


# Probar el laplaciano con la funcion de abajo (cambiar tipo de cell a Code, una vez que haya programado la función order_2() )